# Truncated Singular Value Decomposition (TSVD) Multi-Node Multi-GPU (MNMG) Demo

The TSVD algorithm is a linear dimensionality reduction algorithm that works really well for datasets in which samples correlated in large groups. Unlike PCA, TSVD does not center the data before computation. 

Unlike the single-GPU implementation, The MNMG TSVD API currently requires a Dask cuDF Dataframe as input. `transform()` also returns a Dask cuDF Dataframe. The Dask cuDF Dataframe API is very similar to the Dask DataFrame API, but underlying Dataframes are cuDF, rather than Pandas.

For information on converting your dataset to Dask cuDF format: https://rapidsai.github.io/projects/cudf/en/0.11.0/dask-cudf.html#multi-gpu-with-dask-cudf

For more information about cuML's TSVD implementation: https://rapidsai.github.io/projects/cuml/en/0.11.0/api.html#cuml.dask.decomposition.TruncatedSVD

In [ ]:
import os
import numpy as np

import pandas as pd
import cudf as gd

from cuml.dask.common import to_dask_df
from cuml.dask.datasets import make_blobs

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

from dask_ml.decomposition import TruncatedSVD as skTSVD
from cuml.dask.decomposition import TruncatedSVD as cumlTSVD

## Start Dask Cluster

We can use the `LocalCUDACluster` to start a Dask cluster on a single machine with one worker mapped to each GPU. This is called one-process-per-GPU (OPG). 

In [ ]:
cluster = LocalCUDACluster(threads_per_worker=1)
client = Client(cluster)

## Define Parameters

In [ ]:
n_samples = 2**15
n_features = 128

n_components = 4
random_state = 42

## Generate Data

### GPU

In [ ]:
%%time
X_dcudf, _ = make_blobs(n_samples, 
                       n_features, 
                       centers=2, 
                       cluster_std=1.0,
                       random_state=random_state)

wait(X_dcudf)

### Host

In [ ]:
X_ddf = to_dask_df(X_dcudf).to_dask_array(lengths=True)

## Scikit-learn Model

In [ ]:
%%time
tsvd_sk = skTSVD(n_components=n_components,
                 algorithm="tsqr", 
                 random_state=random_state)

result_sk = tsvd_sk.fit_transform(X_ddf)

## cuML Model

In [ ]:
%%time
tsvd_cuml = cumlTSVD(n_components=n_components,
                     algorithm="full", 
                     n_iter=5000,
                     tol=0.00001,
                     random_state=random_state)

result_cuml = tsvd_cuml.fit_transform(X_dcudf)

## Evaluate Results

### Singular Values

In [ ]:
passed = np.allclose(tsvd_sk.singular_values_, 
                     tsvd_cuml.singular_values_.to_array(), 
                     atol=1e-1)
print('compare tsvd: cuml vs sklearn singular_values_ {}'.format('equal' if passed else 'NOT equal'))

### Components

In [ ]:
sk_components = np.abs(tsvd_sk.components_)
cuml_components = np.abs(np.asarray(tsvd_cuml.components_.as_gpu_matrix()))

passed = np.allclose(sk_components, cuml_components, atol=1e-1)
print('compare tsvd: cuml vs sklearn components_ {}'.format('equal' if passed else 'NOT equal'))

### Transform

In [ ]:
# compare the reduced matrix
passed = np.allclose(result_sk.compute(), np.asarray(result_cuml.compute().as_gpu_matrix()), atol=1)
# larger error margin due to different algorithms: arpack vs full
print('compare tsvd: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal'))